In [24]:
#from pymongo import MongoClient
from bs4 import BeautifulSoup
from multiprocessing import Pool
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
#import progressbar
import requests
import time
import sys
import math
import nltk
from nltk.collocations import *



sources = (("France Info","Politique","https://www.francetvinfo.fr/politique/{}.html"),
          ("France Info","Sante","https://www.francetvinfo.fr/sante/{}.html"),
          ("BFMTV","Sante","https://www.bfmtv.com/sante/page{}/"),
          ("BFMTV","Politique","https://www.bfmtv.com/politique/page{}/"))


def requestLinkWithRetry(link):
    res = None
    tries = 0
    while(tries < 3):
        try:
            res = requests.get(link)
            return res
        except TimeoutError: 
            tries += 1
    return None

# Open the page n depending on the source
def openPage(source,n):
    url = source[2].format(n) # source sans s = le 3 eme élement d'un des sites
    page = requestLinkWithRetry(url)
    soup = BeautifulSoup(page.content,'html.parser')
    return soup

#Get the number of pages
def getNumberPages(source, soup):
    pages = []
    if source[0] == "France Info":
        for span in soup.findAll("span", {"class": "ftvi-btn__label"}):
            number = span.text
            if number.isdigit():
                pages.append(int(number))
        return max(pages)
    else:
        navs = soup.findAll("ul", {"class": "pagination"})
        for nav in navs:
            for a in nav.findAll("a"):
                number = a.text
                if number.isdigit():
                    pages.append(int(number))
        return max(pages)

#Get the links of the articles
def getArticlesLinks(source,soup):
    linksArticles = []
    for a in soup.findAll("article",{"class":"link"}):
        link = soup.find("a")
        linksArticles.append(str(a["href"]))
    return linksArticles

# Get the articles' content je le sors de la demo du cours du prof
def getArticlesFromLinks(links):
    
    #Request all articles in parralel
    p = Pool(10)
    results = p.map(requestLinkWithRetry, links)
    p.close()
    
    #make soup from articles
    soups = []
    for result in progressbar.progressbar(results):
        if(not result is None):
            soups.append(BeautifulSoup(result.content, 'html.parser'))
    return soups

# Get the articles' content
def getFormattedArticles(articles):
    formArt = []


    for a in progressbar.progressbar(articles):
        if (source[0] == "France Info"):
            div= a.find("article",{"class":"page-content content content--classic"})
            title = div.find("h1",{"class":"c-title"}).text
            date = div.find("time",{"class":"c-date"}).text

            


20